In [1]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, State
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Carregar e preparando os dados
df = pd.read_csv("dados/manufacturing_6G_dataset.csv")
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Hour'] = df['Timestamp'].dt.hour

#Realizando o Label Encoding
le_mode = LabelEncoder()
df['Operation_Mode_Code'] = le_mode.fit_transform(df['Operation_Mode'])

le_eff = LabelEncoder()
df['Efficiency_Status_Code'] = le_eff.fit_transform(df['Efficiency_Status'])

#As Features
features = ['Temperature_C', 'Vibration_Hz', 'Power_Consumption_kW',
            'Network_Latency_ms', 'Packet_Loss_%', 'Quality_Control_Defect_Rate_%',
            'Production_Speed_units_per_hr', 'Predictive_Maintenance_Score',
            'Error_Rate_%', 'Operation_Mode_Code']

#Treinando os modelos
X = df[features]
y_eff = df['Efficiency_Status_Code']
y_fail = ((df['Predictive_Maintenance_Score'] > 0.8) | (df['Error_Rate_%'] > 10)).astype(int)

X_train, _, y_train, _ = train_test_split(X, y_eff, test_size=0.2, random_state=42)
model_eff = RandomForestClassifier(n_estimators=50).fit(X_train, y_train)

X_train_f, _, y_train_f, _ = train_test_split(X, y_fail, test_size=0.2, random_state=42)
model_fail = RandomForestClassifier(n_estimators=50).fit(X_train_f, y_train_f)

# ========== APP ==========
app = Dash(__name__)
app.title = "Dashboard Industrial"

# ========== LAYOUT ==========
app.layout = html.Div([
    html.H1("📊 Dashboard Industrial", style={'textAlign': 'center'}),

    dcc.Tabs([
        dcc.Tab(label='Visão Geral', children=[
            html.Div([
                html.Label("Data:"),
                dcc.DatePickerRange(
                    id='filtro-data',
                    min_date_allowed=df['Timestamp'].min().date(),
                    max_date_allowed=df['Timestamp'].max().date(),
                    start_date=df['Timestamp'].min().date(),
                    end_date=df['Timestamp'].max().date()
                ),
                html.Br(), html.Label("Hora do Dia:"),
                dcc.RangeSlider(id='filtro-hora', min=0, max=23, step=1, value=[0, 23],
                                marks={i: str(i) for i in range(0, 24, 4)}),
                html.Br(), html.Label("Máquina:"),
                dcc.Dropdown(
                    id='filtro-maquina',
                    options=[{'label': 'Todas', 'value': 'all'}] +
                            [{'label': str(i), 'value': i} for i in sorted(df['Machine_ID'].unique())],
                    value='all'
                ),
                html.Br(), html.Label("Modo de Operação:"),
                dcc.Dropdown(
                    id='filtro-modo',
                    options=[{'label': 'Todos', 'value': 'all'}] +
                            [{'label': op, 'value': idx} for idx, op in enumerate(le_mode.classes_)],
                    value='all'
                ),
            ], style={'width': '25%', 'display': 'inline-block', 'padding': '20px'}),

            html.Div([
                dcc.Graph(id='grafico-producao'),
                dcc.Graph(id='grafico-erro'),
                dcc.Graph(id='grafico-barras-operacao'),
                dcc.Graph(id='grafico-barras-maquina'),
                dcc.Graph(id='grafico-hist-pmscore')
            ], style={'width': '70%', 'display': 'inline-block', 'verticalAlign': 'top'})
        ]),

        dcc.Tab(label='Predição de Eficiência', children=[
            html.H3("Prever Nível de Eficiência da Máquina"),
            html.Div([
                html.Div([
                    html.Label(f"{f.replace('_', ' ')}:"),
                    dcc.Input(id=f'eff-{f}', type='number', debounce=True, step=0.01)
                ]) for f in features
            ], style={'columnCount': 2}),
            html.Br(),
            html.Button("Prever Eficiência", id="btn-predict-eff"),
            html.Div(id="output-eff", style={'marginTop': '20px', 'fontSize': '20px'})
        ]),

        dcc.Tab(label='Predição de Falha', children=[
            html.H3("Prever Probabilidade de Falha Operacional"),
            html.Div([
                html.Div([
                    html.Label(f"{f.replace('_', ' ')}:"),
                    dcc.Input(id=f'fail-{f}', type='number', debounce=True, step=0.01)
                ]) for f in features
            ], style={'columnCount': 2}),
            html.Br(),
            html.Button("Prever Falha", id="btn-predict-fail"),
            html.Div(id="output-fail", style={'marginTop': '20px', 'fontSize': '20px', 'color': 'red'})
        ])
    ])
])

# Construindo os Callbacks
@app.callback(
    Output('grafico-producao', 'figure'),
    Output('grafico-erro', 'figure'),
    Output('grafico-barras-operacao', 'figure'),
    Output('grafico-barras-maquina', 'figure'),
    Output('grafico-hist-pmscore', 'figure'),
    Input('filtro-data', 'start_date'),
    Input('filtro-data', 'end_date'),
    Input('filtro-hora', 'value'),
    Input('filtro-maquina', 'value'),
    Input('filtro-modo', 'value'),
)
def atualizar_graficos(data_ini, data_fim, horas, maquina, modo):
    dff = df[(df['Timestamp'] >= data_ini) & (df['Timestamp'] <= data_fim)]
    dff = dff[(dff['Hour'] >= horas[0]) & (dff['Hour'] <= horas[1])]
    if maquina != 'all':
        dff = dff[dff['Machine_ID'] == maquina]
    if modo != 'all':
        dff = dff[dff['Operation_Mode_Code'] == modo]
    if dff.empty:
        return [px.scatter(title="Sem dados")] * 5

    fig_prod = px.line(dff, x='Timestamp', y='Production_Speed_units_per_hr', color='Machine_ID', title="Produção")
    fig_erro = px.line(dff, x='Timestamp', y='Error_Rate_%', color='Machine_ID', title="Erro (%)")

    df_mode = dff.groupby('Operation_Mode_Code').agg({'Production_Speed_units_per_hr': 'mean', 'Error_Rate_%': 'mean'}).reset_index()
    df_mode['Mode'] = le_mode.inverse_transform(df_mode['Operation_Mode_Code'])
    fig_bar_op = px.bar(df_mode, x='Mode', y=['Production_Speed_units_per_hr', 'Error_Rate_%'], barmode='group', title='Produção e Erro por Modo')

    df_machine = dff.groupby('Machine_ID').agg({'Production_Speed_units_per_hr': 'mean', 'Error_Rate_%': 'mean'}).reset_index()
    fig_bar_mach = px.bar(df_machine, x='Machine_ID', y=['Production_Speed_units_per_hr', 'Error_Rate_%'], barmode='group', title='Por Máquina')

    fig_hist = px.histogram(dff, x='Predictive_Maintenance_Score', nbins=30, title='Distribuição Score de Manutenção')

    return fig_prod, fig_erro, fig_bar_op, fig_bar_mach, fig_hist

#Callbacks
@app.callback(
    Output("output-eff", "children"),
    Input("btn-predict-eff", "n_clicks"),
    [State(f'eff-{f}', 'value') for f in features],
    prevent_initial_call=True
)
def prever_eficiencia(n, *valores):
    if None in valores:
        return "Por favor, preencha todos os campos."
    pred = model_eff.predict([valores])[0]
    classe = le_eff.inverse_transform([pred])[0]
    return f"🔍 Eficiência prevista: {classe}"

#Callbacks
@app.callback(
    Output("output-fail", "children"),
    Input("btn-predict-fail", "n_clicks"),
    [State(f'fail-{f}', 'value') for f in features],
    prevent_initial_call=True
)
def prever_falha(n, *valores):
    if None in valores:
        return "Por favor, preencha todos os campos."
    prob = model_fail.predict_proba([valores])[0][1]
    return f"🚨 Probabilidade de falha: {prob*100:.1f}%"

#Para rodar o app, execute isto
if __name__ == '__main__':
    app.run_server(debug=True)


c:\Users\Matt\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names

c:\Users\Matt\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names

c:\Users\Matt\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names

c:\Users\Matt\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names

c:\Users\Matt\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have vali